In [1]:
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
import lseg.data as ld  # Install with `pip install lseg.data`
ld.open_session()

<lseg.data.session.Definition object at 0x16799e5c0 {name='workspace'}>

In [3]:
s_and_p_companies = ld.get_data(
    "0#.SPX",
    fields=["TR.CommonName"]
)["Instrument"]
print(s_and_p_companies)


0      POOL.OQ
1      CHRW.OQ
2        AJG.N
3        CNP.N
4       AMCR.N
        ...   
498      PSX.N
499     SCHW.N
500    SNPS.OQ
501        J.N
502      AVY.N
Name: Instrument, Length: 503, dtype: string


In [7]:
first = s_and_p_companies[0]
request_string = f"R:{first} AND Language:LEN AND Source:RTRS"
df = ld.news.get_headlines(
    request_string,
    start="2023-03-01",
    end="2024-03-01",
    count = 5000
    )
len(df)

17

In [5]:
df

,headline,storyId,sourceCode
versionCreated,,,


In [106]:
for idx, storyId in enumerate(df['storyId'].values):  # for each row in our df dataframe
    newsText = None
    try:
        newsText = ld.news.get_story(storyId, format=ld.news.Format.TEXT)  # get the
    except Exception as e:
        print(f"Error getting story {storyId}: {e}")
        continue
    if newsText:
        pass
df.head()

,headline,storyId,sourceCode
versionCreated,,,
2025-02-28 19:22:11.554,"THE NASDAQ PRE-MARKET BEARISH STOCK SNIPPETS POOL (POOL: $342.54) Friday February 28, 2025 07:02...",urn:newsml:newsroom:20250228:nNRAvks33f:0,NS:GLOBEA
2025-02-28 19:12:03.749,Pool (POOL: $342.78) lifts ; +24c [0.1%],urn:newsml:newsroom:20250228:nNRAvkrz5a:0,NS:GLOSTO
2025-02-28 18:52:23.834,Pool (POOL: $343.67) increases $1.13; +$1.13 [0.3%],urn:newsml:newsroom:20250228:nNRAvkrrx5:0,NS:GLOSTO
2025-02-28 18:35:10.036,German trading: Pool (SP1: EUR329.0) decreases on thin volume; -EUR1.70 [-0.5%],urn:newsml:newsroom:20250228:nNRAvkriew:0,NS:GERCOM
2025-02-28 18:17:10.222,Pool (POOL: $343.67) increases $1.13; +$1.13 [0.3%],urn:newsml:newsroom:20250228:nNRAvkr6y6:0,NS:GLOSTO


In [8]:
import datetime
from datetime import timedelta
from tqdm import tqdm  # for progress bars
import pandas as pd

s_and_p_companies = ld.get_data(
    "0#.SPX",
    fields=["TR.CommonName"]
)["Instrument"]
print(s_and_p_companies)

# 3) Define your years of interest
years = range(2023, 2026)

print(s_and_p_companies)

0      POOL.OQ
1      CHRW.OQ
2        AJG.N
3        CNP.N
4       AMCR.N
        ...   
498      PSX.N
499     SCHW.N
500    SNPS.OQ
501        J.N
502      AVY.N
Name: Instrument, Length: 503, dtype: string
0      POOL.OQ
1      CHRW.OQ
2        AJG.N
3        CNP.N
4       AMCR.N
        ...   
498      PSX.N
499     SCHW.N
500    SNPS.OQ
501        J.N
502      AVY.N
Name: Instrument, Length: 503, dtype: string


In [213]:
# 4) Loop over each year, retrieve news, and store to year-based CSV
for year in years:
    print(f"Processing year {year} ...")

    # Build date range
    start_date_str = f"{year}-01-01"
    end_date_str   = f"{year}-12-31"

    # Prepare a container for all news data in this year
    all_news_records = []

    # 5) Loop over each S&P company with a progress bar
    for company_ric in tqdm(s_and_p_companies, desc=f"Year {year}"):
        # Build the query for English news about this RIC
        query_string = f"R:{company_ric} AND Language:LEN AND Source:RTRS"

        try:
            headlines_df = ld.news.get_headlines(
                query=query_string,
                start=start_date_str, 
                end=end_date_str, 
                count=10000
            )
        except Exception as e:
            print(f"Error fetching headlines for {company_ric} in {year}: {e}")
            continue

        if headlines_df is None or headlines_df.empty:
            continue

        # 6) Fetch full story text for each headline
        for idx, row in headlines_df.iterrows():
            story_id = row["storyId"]
            version_created = row.name
            headline_text = row.get("headline", "")

            # Attempt to get the news story text
            try:
                story_text = ld.news.get_story(story_id, format=ld.news.Format.TEXT)
            except Exception as e:
                # print(f"Error retrieving story {story_id} for {company_ric}: {e}")
                continue

            # Build one record (row) of data
            news_record = {
                "year": year,
                "company_ric": company_ric,
                "storyId": story_id,
                "versionCreated": version_created,
                "headline": headline_text,
                "newsText": story_text,
            }
            all_news_records.append(news_record)

    # 7) Convert to DataFrame and save to CSV
    year_df = pd.DataFrame(all_news_records)

    # For large data, consider writing in chunks or appending 
    # to avoid memory issues. For demonstration, we do it in one go:
    csv_filename = f"{year}.csv"
    year_df.to_csv(csv_filename, index=False)
    print(f"Saved {len(year_df)} news articles to {csv_filename}")


Processing year 2023 ...


Year 2023:  50%|████▉     | 250/503 [47:17<24:27,  5.80s/it]    

Error fetching headlines for IT.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for ECL.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for BXP.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for GE.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for ED.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for ALLE.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for OTIS.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for WFC.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for CTVA.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for FTV.N in 2023: Error code 429 | Too

Year 2023:  60%|█████▉    | 301/503 [47:17<06:41,  1.99s/it]

Error fetching headlines for O.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for MLM.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for RVTY.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for L.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for QCOM.OQ in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for IPG.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for PHM.N in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for ANSS.OQ in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for ULTA.OQ in 2023: Error code 429 | Too many requests, please try again later.
Error fetching headlines for KO.N in 2023: Error code 429 | T

...

Error fetching headlines for V.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for YUM.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for LYV.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for CPB.OQ in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for JNPR.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for EG.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for FTNT.OQ in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for DTE.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for HST.OQ in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for IRM.N in 2025: Error code 429 | T

Year 2025:  99%|█████████▉| 499/503 [00:02<00:00, 197.94it/s]

Error fetching headlines for SO.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for LII.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for EQT.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for NRG.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for MKTX.OQ in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for APO.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for LMT.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for DLR.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for PSX.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for SCHW.N in 2025: Error code 429 | T

Year 2025: 100%|██████████| 503/503 [00:02<00:00, 214.81it/s]

Error fetching headlines for J.N in 2025: Error code 429 | Too many requests, please try again later.
Error fetching headlines for AVY.N in 2025: Error code 429 | Too many requests, please try again later.
Saved 0 news articles to 2025.csv


In [28]:
import datetime
from datetime import timedelta
import time
from tqdm import tqdm  # for progress bars
import pandas as pd
import os
import logging

# Configure logging to suppress HTTP request logs
logging.getLogger('urllib3').setLevel(logging.WARNING)
logging.getLogger('requests').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('refinitiv.data').setLevel(logging.WARNING)
logging.getLogger('lseg.data').setLevel(logging.WARNING)

# 1. Create directories if they don't exist
years = range(2023, 2026)
for year in years:
    os.makedirs(f"news_data/{year}", exist_ok=True)

# 2. Load tracking file if it exists, or create a new one
tracking_file = "processed_companies.csv"
if os.path.exists(tracking_file):
    processed_df = pd.read_csv(tracking_file)
    processed_companies = set(processed_df["company_ric"].tolist())
    print(f"Loaded {len(processed_companies)} already processed companies")
else:
    processed_companies = set()
    print("No tracking file found. Starting fresh.")

# Filter out already processed companies
companies_to_process = [company for company in s_and_p_companies if company not in processed_companies]

# 4. Process each company (all years) with a progress bar
for company_ric in tqdm(companies_to_process, desc="Processing companies", 
                        bar_format='{l_bar}{bar:30}| {n_fmt}/{total_fmt} companies [{elapsed}<{remaining}]'):
    company_fully_processed = True
    company_data_by_year = {year: [] for year in years}
    
    # Create uppercase RIC for filenames
    company_ric_uppercase = company_ric.upper()

    if company_ric_uppercase == "AJG.N":
        print(f"Skipping {company_ric} due to API issue")
        continue
    
    # Process each year for this company
    for year in years:
        
        # Build date range
        start_date_str = f"{year}-01-01"
        end_date_str = f"{year}-12-31"
        
        # Build the query for English news about this RIC
        query_string = f"R:{company_ric} AND Language:LEN AND Source:RTRS"
        
        try:
            headlines_df = ld.news.get_headlines(
                query=query_string,
                start=start_date_str,
                end=end_date_str,
                count=10000
            )
            
            if headlines_df is None or headlines_df.empty:
                print(f"No headlines found for {company_ric} in {year}")
                continue
            else:
                print(f"Found {len(headlines_df)} headlines for {company_ric} in {year}")
            
            # Enhanced progress bar for headlines
            # Process each headline with a more detailed progress bar
            for idx, row in tqdm(
                headlines_df.iterrows(),
                desc=f"{company_ric} - {year}",
                total=len(headlines_df),
                leave=False,
                bar_format='{desc}: {percentage:3.0f}%|{bar:30}| {n_fmt}/{total_fmt} headlines [{elapsed}<{remaining}, {rate_fmt}{postfix}]',
                unit='headline'
            ):
                story_id = row["storyId"]
                version_created = row.name
                headline_text = row.get("headline", "")
                
                # Attempt to get the news story text
                try:
                    story_text = ld.news.get_story(story_id, format=ld.news.Format.TEXT)
                    
                    # Skip records where story_text is NaN or None
                    if pd.isna(story_text) or story_text is None or story_text == "":
                        continue
                    
                    # Build one record (row) of data
                    news_record = {
                        "year": year,
                        "company_ric": company_ric,
                        "storyId": story_id,
                        "versionCreated": version_created,
                        "headline": headline_text,
                        "newsText": story_text,
                    }
                    company_data_by_year[year].append(news_record)
                    
                except Exception as e:
                    # If we hit rate limit, mark company as not fully processed
                    if "429" in str(e):
                        company_fully_processed = False
                        print(f"Rate limit reached while processing story {story_id} for {company_ric} in {year}: {e}")
                        break
                    continue
            
            # Check if we hit rate limit and need to stop
            if not company_fully_processed:
                break
                
        except Exception as e:
            # If we hit rate limit, mark company as not fully processed
            if "429" in str(e):
                company_fully_processed = False
                print(f"Rate limit reached for {company_ric} in {year}: {e}")
                break
            print(f"Error fetching headlines for {company_ric} in {year}: {e}")
            continue
    
    # Only save data if the company was fully processed
    if company_fully_processed:
        # Save data for each year
        for year in years:
            if company_data_by_year[year]:  # Only save if there's data
                year_df = pd.DataFrame(company_data_by_year[year])
                csv_filename = f"news_data/{year}/{company_ric_uppercase}.csv"
                year_df.to_csv(csv_filename, index=False)
                print(f"Saved {len(year_df)} news articles for {company_ric} in {year} to {csv_filename}")
        
        # Add to processed companies
        processed_companies.add(company_ric)
        
        # Update tracking file
        pd.DataFrame({"company_ric": list(processed_companies)}).to_csv(tracking_file, index=False)
        print(f"✓ {company_ric} fully processed and saved to tracking file")
    else:
        print(f"⨯ {company_ric} was NOT fully processed. No data was saved.")
    
    # If we hit rate limit, stop processing additional companies
    if not company_fully_processed:
        print("API rate limit reached. Stopping processing.")
        break

print("Processing complete.")

Loaded 135 already processed companies


Processing companies:   0%|                              | 0/368 companies [00:00<?]

Skipping AJG.N due to API issue


Processing companies:   0%|                              | 1/368 companies [00:01<09:29]

Rate limit reached for JNJ.N in 2023: Error code 429 | Too many requests, please try again later.
⨯ JNJ.N was NOT fully processed. No data was saved.
API rate limit reached. Stopping processing.
Processing complete.
